In [42]:
# Many functions within this notebook were developed using the available online tensorflow tutorial and then modified for personal use and flexibility
    # https://www.tensorflow.org/tutorials/structured_data/time_series

# Libraries for the rest of the code
import pandas as pd
import os
import numpy as np
from tkinter import *
import json
import pickle
import matplotlib.pyplot as plt

import tensorflow as tf
#from keras.utils import generic_utils
#from tensorflow.keras import backend as K
from tensorflow.keras.layers import LSTM, Dense, GRU, RNN, Bidirectional, Dropout
from platform import python_version
from tensorflow.keras.preprocessing import timeseries_dataset_from_array 
#from tensorflow import keras as ks

from tensorflow.keras import Input, Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

# Print tf version
print('TF version:' ,tf.version.VERSION, '\n')

TF version: 2.10.0 



In [2]:
# Check if GPU is detected and check cuda (1) gpu device type (2) compute capability details
list_device = [(x, tf.config.experimental.get_device_details(x)) for x in tf.config.list_physical_devices('GPU')]
# Check if TF package is built with Cuda support
tf_built_with_cuda_support = tf.test.is_built_with_cuda()
tf_built_with_gpu_support = tf.test.is_built_with_gpu_support()

print("GPUs Detected: ", len(tf.config.list_physical_devices('GPU')))
print('Current device:', list_device)
print('TF built with GPU support:', tf_built_with_gpu_support)
print('TF built with cuda support:', tf_built_with_cuda_support, '\n')

# Further test gpu by a simple addition on the gpu device
class MyTest(tf.test.TestCase):

    def test_add_on_gpu(self):
        if not tf.test.is_built_with_gpu_support():
            self.skipTest("test is only applicable on GPU")
            print('GPU is not found!')
        with tf.device("GPU:0"):
            self.assertEqual(tf.math.add(1.0, 2.0), 3.0)
            print('GPU is available!')

# Run the test on gpu
myinstance = MyTest()
myinstance.test_add_on_gpu()

# If gpu is seen, check the memory usage by TF
if len(tf.config.list_physical_devices('GPU')) > 0: 
    print('\nCurrent GPU usage by TF:', tf.config.experimental.get_memory_info('GPU:0'))

GPUs Detected:  1
Current device: [(PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), {'device_name': 'NVIDIA GeForce RTX 4060 Laptop GPU', 'compute_capability': (8, 9)})]
TF built with GPU support: True
TF built with cuda support: True 

GPU is available!

Current GPU usage by TF: {'current': 2048, 'peak': 2560}


In [3]:
# Window Generator class to control input and outputs (processes each well separately and combines batches later as long as well_names are present in provided excel sheet)

# WindowGenerator class for organizing input-output structure of data for training
class WindowGenerator():
    def __init__(self, input_width, label_width, 
                 shift, input_columns=None, label_columns=None,shuffle_data = False,batch_number = 32,
                 train_df=None, val_df=None, test_df=None): 
        
        # Normalized data
        self.train_df_norm = train_df[input_columns]
        self.val_df_norm = val_df[input_columns]
        self.test_df_norm = test_df[input_columns]
        
        # Preprocessed raw dataset (includes normalized input columns)
        self.train_df_raw = train_df  
        self.val_df_raw = val_df
        self.test_df_raw = test_df
        
        # Shuffling and batch size options in make_dataset method
        self.shuffle_data = shuffle_data
        self.batch_number = batch_number

        # Work out the label column indices.
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in
                                        enumerate(label_columns)}
        
        self.column_indices = {name: i for i, name in
                               enumerate(self.train_df_norm.columns)}
        
        # Initialize user given input columns
        self.input_columns = input_columns

        # Work out the window parameters.
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def __repr__(self):
        return '\n'.join([
            f'Total window size: {self.total_window_size}',
            f'Input indices: {self.input_indices}',
            f'Label indices: {self.label_indices}',
            f'Label column name(s): {self.label_columns}'])
    
    # Split time series data into windows (segments) of input and user specified time-shifted output (label) 
    def split_window(self, features):
        inputs = features[:, self.input_slice, :]
        labels = features[:, self.labels_slice, :]
        if self.label_columns is not None:
            labels = tf.stack(
                [labels[:, :, self.column_indices[name]] for name in self.label_columns],
                axis=-1)

        # Slicing doesn't preserve static shape information, so set the shapes manually.
        # This way the `tf.data.Datasets` are easier to inspect.
        inputs.set_shape([None, self.input_width, None])
        labels.set_shape([None, self.label_width, None])

        return inputs, labels


    
    # For making time series data sets
    def make_dataset(self, data):
        data = np.array(data, dtype=np.float32)
        ds = tf.keras.utils.timeseries_dataset_from_array(
            data=data,
            targets=None,
            sequence_length=self.total_window_size,
            sequence_stride=1,
            shuffle=self.shuffle_data,
            batch_size=self.batch_number,)
        
            
        # Mapped ds (mapped to input, output based on the window size and label size)
        ds = ds.map(self.split_window)

        # Separate inputs and outputs
        inputslist = []
        outputslist = []
        for index,input_batch in enumerate(ds):
            inputslist.append(input_batch[0])
            outputslist.append(input_batch[1])

        # convert inputs and outputs to tf tensors
        tf_inputs = tf.concat(inputslist,axis=0)
        tf_outputs = tf.concat(outputslist,axis=0)

        # Save tf_inputs and tf_outputs as tf datasets
        tf_inputs_dataset = tf.data.Dataset.from_tensor_slices(tf_inputs)
        tf_outputs_dataset = tf.data.Dataset.from_tensor_slices(tf_outputs)

        # Combine inputs and outputs
        model_inputs = tf.data.Dataset.zip(((tf_inputs_dataset), tf_outputs_dataset))
        model_inputs = model_inputs.batch(self.batch_number).cache().prefetch(buffer_size=tf.data.AUTOTUNE)

        return model_inputs

    
    # For accesing training, validation, and test data sets and creating example datasets from train data set
    @property
    def train(self):
        
        self.full_batch = []
        for i, current_well in enumerate(self.train_df_raw['API/UWI List'].unique()):
            self.full_batch.append(self.make_dataset(self.train_df_raw.loc[self.train_df_raw['API/UWI List'] == current_well][self.input_columns]))
             
        if len(self.full_batch) == 1:
            ds = self.full_batch[0]
        elif len(self.full_batch) == 2:
            ds = self.full_batch[0].concatenate(self.full_batch[1])
        elif len(self.full_batch) > 2:
            ds = self.full_batch[0].concatenate(self.full_batch[1])
            for i,name in enumerate(self.full_batch):
                if i >= 2:
                    ds = ds.concatenate(self.full_batch[i])
        return ds

    @property
    def val(self):
        
        self.full_batch = []
        for i, current_well in enumerate(self.val_df_raw['API/UWI List'].unique()):
            self.full_batch.append(self.make_dataset(self.val_df_raw.loc[self.val_df_raw['API/UWI List'] == current_well][self.input_columns]))
             
        if len(self.full_batch) == 1:
            ds = self.full_batch[0]
        elif len(self.full_batch) == 2:
            ds = self.full_batch[0].concatenate(self.full_batch[1])
        elif len(self.full_batch) > 2:
            ds = self.full_batch[0].concatenate(self.full_batch[1])
            for i,name in enumerate(self.full_batch):
                if i >= 2:
                    ds = ds.concatenate(self.full_batch[i])
        
        return ds

    @property
    def test(self):
        
        self.full_batch = []
        for i, current_well in enumerate(self.test_df_raw['API/UWI List'].unique()):
            self.full_batch.append(self.make_dataset(self.test_df_raw.loc[self.test_df_raw['API/UWI List'] == current_well][self.input_columns]))
             
        if len(self.full_batch) == 1:
            ds = self.full_batch[0]
        elif len(self.full_batch) == 2:
            ds = self.full_batch[0].concatenate(self.full_batch[1])
        elif len(self.full_batch) > 2:
            ds = self.full_batch[0].concatenate(self.full_batch[1])
            for i,name in enumerate(self.full_batch):
                if i >= 2:
                    ds = ds.concatenate(self.full_batch[i])
        
        return ds


In [4]:
# Allows the user to select from multiple excel sheets in a given excel file
    ## Select the desired sheet or sheets one-by-one and load once finished selecting

# This allows some flexibility to keep your data in separate excel sheets and experiment with the models you are developing
    ## For example: 
        ### There are multiple different formations the wells are producing from and you would like to keep them separate
        ### You may develop separate models for each formation (separated using different excel sheets)
        ### Alternatively, you may develop a single model that can learn to differentiate and handle wells producing from different formations
        ### The choice in how you want to handle model development is completely up to you.

# Warning: Selecting "all" will select all the sheets. Make sure the selected excel sheets are in the correct format. 
# Warning: This will not work with multiple excel files --> It will work with different excel sheets. You can modify this behavior if needed.

def select_sheets(sheet_names = None):
    
    window = Tk()
    window.title('Multiple selection')
    
    user_selected_sheets = []

    # for scrolling vertically
    yscrollbar = Scrollbar(window)
    yscrollbar.pack(side = RIGHT, fill = Y)

    label = Label(window,
                  text = "Select the available sheets below :  ",
                  font = ("Times New Roman", 10), 
                  padx = 10, pady = 10)
    label.pack()
    listbox = Listbox(window, selectmode = "multiple", 
                   yscrollcommand = yscrollbar.set)

    # Widget expands horizontally and vertically by assigning both to fill option
    listbox.pack(padx = 10, pady = 10,
              expand = YES, fill = "both")

    x = sheet_names

    for each_item in range(len(x)):

        listbox.insert(END, x[each_item])
        listbox.itemconfig(each_item, bg = "white")

    # Attach listbox to vertical scrollbar
    yscrollbar.config(command = listbox.yview)

    def selected_item():
        # Traverse the tuple returned by the selection method and print corresponding value(s) in the listbox
        for i in listbox.curselection():
            user_selected_sheets.append(listbox.get(i)) 
        print('Selected sheets:')
        print(user_selected_sheets) # listbox.get(i)
        window.destroy()
    # Create a button widget and map the command parameter to selected_item function
    btn = Button(window, text='Create List', command=selected_item)

    # Placing the button and listbox
    btn.pack(side='bottom')
    listbox.pack()

    window.mainloop()

    return user_selected_sheets

In [5]:
# Preprocess data after importing it
def process_data(Production_data = None, train_percent = None, val_percent = None, test_percent = None, label_columns = None, control_parameters = None):

    # Process data before splitting
    Data_copy = Production_data

    store_wells = []
    wells_df = pd.DataFrame(columns=Data_copy.columns)
    count = 0
    for index,well_name in enumerate(Data_copy['API/UWI List'].unique()):

        # Find current well
        current_well = Data_copy.loc[Data_copy['API/UWI List'] == well_name].reset_index(drop=True)
        # full production history
        full_indices = len(current_well)
        
        # Processed removed production history - Removes the indices before maximum oil rate in a given well.
        # Assumes that the oil phase is the stable phase and that the overall production rate starts declining after reaching the maximum oil rate.
        max_index_oil = current_well['Monthly Oil'].idxmax()
        current_well = current_well[max_index_oil:]
        removed_indices = len(current_well)

        # Remove wells based on potential index length to be able to select larger window sizes for training, validation, testing
        train_indices = int(removed_indices*train_percent)
        val_indices = int(removed_indices*val_percent)
        test_indices = int(removed_indices*test_percent)

        # If (window size >= 20) or more than 6 timesteps removed for removing indices < qo_max (maximum oil rate)
        # (full_indices - removed_indices > 6) --> this statement checks the number of indices removed previously (if more than 6 indices were removed it assumes that the well behavior was erratic enough not to be included in the training or testing)
        # for val_indices and test_indices, this checks whether there are at least 12 discrete points to validate and test the developed model with
            
            ## Reason # 1: The training is "moving window" based and the window for training and testing was selected to include 6 discrete points. Therefore, when evaluating, if there are 12 points, the window will go through the validation and test sets at least 6 times or more
            ## Reason # 2: Wanted to be sure that the model performance evaluation is trustworthy: when tested on lower than 12 points, there is no guarantee that the model is stable enough during hindcasting even if it shows high accuracy.
                    #### Ideal scenario: the more data points available for validation and testing, the more trustworthy the model performance evaluation becomes (restricted due to data availability). 
                ### The Window size should be user defined based on available data for training, validation, and testing.
                    #### Ideal scenario: We can select a window size greater than 6 due to high resolution (daily, weekly, etc.) or wells that have been producing longer.
                    #### Hint: In a many-to-many or sequence-to-sequence type training, each discrete point's error is determined independently and then accumulated. Therefore, having longer sequences can help train a more accurate model by encouraging the model to minimize the additional errors coming from the additional data points.
                ### Hindcasting: when testing for the same well for validating the model's ability to predict future production in the direction of time 
        if (full_indices - removed_indices > 6) or (test_indices) < 12 or (val_indices) < 12:
            count = count + 1
            current_well = pd.DataFrame() # Ignores the well by creating an empty dataframe if the above conditions are not met

        # If current well is not empty after processing, store the well
        if not current_well.empty:
            # Replace all '0' with selected number
            current_well[label_columns] = current_well[label_columns].replace(0, 0.1)
            
            # Shift Days column up (Production days on per month) and set days column as future control parameters (days @ timestep= t+1, monthly production @ timestep = t)
            current_well = current_well.copy()
            current_well[control_parameters] = current_well[control_parameters].shift(-1)
            current_well = current_well.dropna().reset_index(drop=True) # Drop NA rows after shifting control parameters (days) up
            store_wells.append(current_well)
        

    # Combine all remaining wells (after preprocessing) and print the results for the user to see 
    wells_df = pd.concat(store_wells).reset_index(drop=True)    
    
    print('Number of wells in original data:', len(Data_copy['API/UWI List'].unique()))
    print('Number of wells removed:', count)
    print('Remaining number of wells:', len(wells_df['API/UWI List'].unique()))
    
    return wells_df


In [6]:
# Controls the splitting of production data into training, validation, and test sets.
# Assumes the split should be temporal to apply hindcasting to the model.
    ### Hindcasting: when testing for the same well for validating the model's ability to predict future production in the direction of time

def split_data(DF_WELL,train_percent=0.7,val_percent=0.2,test_percent=0.1):

    total_percent = round((train_percent + val_percent + test_percent),2)

    if total_percent != 1:
        raise Exception("Total percentage of train,val,test must be equal to 100%")
    else:
        train_data = []
        val_data = []
        test_data = []

        # Slicing for rest of the data after train and val (test data)
        train_and_val = train_percent + val_percent # total percent of train + val


        # Extract desired input features and split data into train,val,test
        for index, current_well in enumerate(DF_WELL['API/UWI List'].unique()):

            unique_well = DF_WELL.loc[DF_WELL['API/UWI List'] == current_well]
            n = len(unique_well)
            train_data.append(unique_well.iloc[0:int(n*train_percent)])
            val_data.append(unique_well.iloc[int(n*train_percent):int(n*train_and_val)])
            test_data.append(unique_well.iloc[int(n*train_and_val):])

        train_df = pd.concat(train_data)
        val_df = pd.concat(val_data)
        test_df = pd.concat(test_data)
        
        return train_df, val_df, test_df

# Normalize dynamic variables in train, val, test data sets using z-score normalization
def normalize_data(train_df, val_df, test_df, plot_cols = ['BP', 'BHP', 'OPR'], norm_method = 'z-score'):
    
    train_mu = []
    train_sigma = []
    train_maximum = []
    train_minimum = []
    train_df_norm = []
    val_df_norm = []
    test_df_norm = []
    
    if norm_method == 'z-score':
        # Save mean per well
        for index, current_well in enumerate(train_df['API/UWI List'].unique()):

            train_unique_well = train_df.loc[train_df['API/UWI List'] == current_well]
            val_unique_well = val_df.loc[val_df['API/UWI List'] == current_well]
            test_unique_well = test_df.loc[test_df['API/UWI List'] == current_well]
            train_mean = train_unique_well[plot_cols].mean()
            train_std = train_unique_well[plot_cols].std()
            train_mu.append(train_mean)
            train_sigma.append(train_std)

            # Normalize data
            train_df_norm.append((train_unique_well[plot_cols]-train_mean)/train_std)
            val_df_norm.append((val_unique_well[plot_cols]-train_mean)/train_std)
            test_df_norm.append((test_unique_well[plot_cols]-train_mean)/train_std)

        for index,value in enumerate(train_df_norm):
            train_df_norm[index] = train_df_norm[index].add_suffix('_norm')
            val_df_norm[index] = val_df_norm[index].add_suffix('_norm')
            test_df_norm[index] = test_df_norm[index].add_suffix('_norm')

        train_df_norm = pd.concat(train_df_norm)
        val_df_norm = pd.concat(val_df_norm)
        test_df_norm = pd.concat(test_df_norm)

        train_df = pd.concat([train_df,train_df_norm],axis=1)
        val_df = pd.concat([val_df,val_df_norm],axis=1)
        test_df = pd.concat([test_df,test_df_norm],axis=1)

        return train_df, val_df, test_df, train_mu, train_sigma
    
    elif norm_method == 'min-max':
        
        # Save mean per well
        for index, current_well in enumerate(train_df['API/UWI List'].unique()):

            train_unique_well = train_df.loc[train_df['API/UWI List'] == current_well]
            val_unique_well = val_df.loc[val_df['API/UWI List'] == current_well]
            test_unique_well = test_df.loc[test_df['API/UWI List'] == current_well]
            train_max = train_unique_well[plot_cols].max()
            train_min = 0 # train_unique_well[plot_cols].min()
            train_maximum.append(train_max)
            train_minimum.append(train_min)

            # Normalize data
            train_df_norm.append((train_unique_well[plot_cols]-train_min)/(train_max-train_min))
            val_df_norm.append((val_unique_well[plot_cols]-train_min)/(train_max-train_min))
            test_df_norm.append((test_unique_well[plot_cols]-train_min)/(train_max-train_min))

        for index,value in enumerate(train_df_norm):
            train_df_norm[index] = train_df_norm[index].add_suffix('_norm')
            val_df_norm[index] = val_df_norm[index].add_suffix('_norm')
            test_df_norm[index] = test_df_norm[index].add_suffix('_norm')

        train_df_norm = pd.concat(train_df_norm)
        val_df_norm = pd.concat(val_df_norm)
        test_df_norm = pd.concat(test_df_norm)

        train_df = pd.concat([train_df,train_df_norm],axis=1)
        val_df = pd.concat([val_df,val_df_norm],axis=1)
        test_df = pd.concat([test_df,test_df_norm],axis=1)

        return train_df, val_df, test_df, train_maximum, train_minimum
        
        
    
    # If desired, user can develop their own custom normalization method    
    elif norm_method == 'custom-method':
        print('not available yet...try another normalization method...')


In [7]:
# Load the model and print model summary and type of model

# find the type of model loaded (integrated as part of load model function)
def find_model_type(model = None):
    
    model_type = []
    # Loop through each layer of the model to find model type
    for layer in model.layers:
        if 'Bidirectional' in str(type(layer)):
            model_type.append('Bidirectional')
            if 'GRU' in str(layer.layer):
                model_type.append('GRU')
            elif 'LSTM' in str(layer.layer):
                model_type.append('LSTM')
        elif 'GRU' in str(type(layer)):
            model_type.append('GRU')
        elif 'LSTM' in str(type(layer)):
            model_type.append('LSTM')
        elif 'Dense' in str(type(layer)):
            model_type.append('Dense')
    
    model_type = list(set(model_type))

    return model_type

# Load the model 
def load_model(model_path = None):
    import json
    
    loaded_model = tf.keras.models.load_model(model_path) # ,custom_objects={"tf": tf}
    # Print model type
    model_type = find_model_type(model = loaded_model)
    print('Model Type:')
    print(model_type)
    
    # Print model summary
    print('\nModel Summary:')
    loaded_model.summary()
    
    # Print optimizer configs
    print('\nOptimizer configs:')
    print(loaded_model.optimizer.get_config())
    
    # Print units for each layer
    print('\nLayer units:')
    for item in loaded_model.get_config()['layers']:
        while 'layer' in item['config']:
            item = item['config']['layer']
        item  = item['config']
        if 'units' in item:
            print('units: ',item['units'])
    
    # If a trainig history exists, print the saved training and validation errors (MAE and MAPE)
    # When saving the errors, they are saved as a json file
    print('\nTraining History:')
    if os.path.exists(os.path.join(model_path,'history.json')):
        history_dict = json.load(open(os.path.join(model_path,'history.json'), 'r'))
    
        if 'loss' in history_dict.keys():
            print('Epochs: ', len(history_dict['loss']))
            print('Train-Loss: ', history_dict['loss'][-1])
        if 'val_loss' in history_dict.keys():
            print('Val-Loss: ', history_dict['val_loss'][-1])
        if 'mae' in history_dict.keys():
            print('Train-MAE: ', history_dict['mae'][-1])
        if 'val_mae' in history_dict.keys():
            print('Val-MAPE: ', history_dict['val_mae'][-1])
        if 'mape' in history_dict.keys():
            print('Train-MAPE: ', history_dict['mape'][-1])
        if 'val_mape' in history_dict.keys():
            print('Val-MAPE: ', history_dict['val_mape'][-1])
    else:
        history_dict = []
        print(history_dict)
    

In [8]:
# # Functions for saving and calculating losses per well

def loss_compute(y_label = None, y_pred = None, y_label_full = [None], loss_type = 'mse'):
    
    """This function is a general function that calculates losses using the possible list of loss selections: 
    ['mse','rmse','mae','mape','nmse','nrmse','nmape','nmae','wmape']
     
     Inputs:
     *y_label = ground truth values (array like) 
     *y_pred = predicted values (array like)
     *y_label_full = ground truth values (array like) -> This can be either the full dataset or partial dataset (e.g., train,val and test)
     *loss_type = type of desired loss (str or list(str)) --> multiple loss types can be requested in list(str) (e.g. ['mse','rmse','mae'])
         **List of possible loss_type options: ['mse','rmse','mae','mape','nmse','nrmse','nmape','nmae','wmape']
     
     Outputs:
     * losses = A dictionary of losses
     
     EXPLANATION OF y_label_full: 
     y_label_full is used in calculating Interquartile Range (IQR or 50% percentile).
     
     EXPLANATION OF IQR: 
     IQR is integrated into some of the loss metrics such as [nmse,nrmse,nmae] to normalize [mse,rmse,mae] losses.
     It allows for more accurate representation of the errors when extreme outliers exist in the dataset. 
     It is also said to be a better normalized error metric that more accurately represents the robustness of the model in case of existing outliers.
     If y_label = train_ground_truth, it is recommended to set y_label_full = train_ground_truth, etc. but, the full dataset can also be used by the user if desired.
     """
    
    # Import inside function to sort created loss dictionary in alphabetical order of key items
    from collections import OrderedDict
    
    # *** Initialize the variables ***
        
    # Possible selections list for loss_type
    possible_selections = ['mse','rmse','mae','mape','nmse','nrmse','nmape','nmae','wmape']
    
    # *** CATCH POSSIBLE ERRORS ***
    # Raise Syntax Error if loss_type is not of type 'str'
    if (not isinstance(loss_type, list)) & (not isinstance(loss_type, str)): 
        raise TypeError('loss_type is not of type str: {}.\nList of valid options: {}'.format(loss_type, possible_selections))
    # Raise Syntax Error if loss_type is not in possible_selections list
    elif (not isinstance(loss_type, list)):
        if (loss_type.lower() not in possible_selections):
            raise SyntaxError('Provided loss_type argument is not a valid argument: {}.\nList of valid options: {}'.format(loss_type, possible_selections))
    elif (isinstance(loss_type, list)) & (any(not isinstance(item, str) for item in loss_type)):
        raise TypeError('an element in loss_type list is not of type str: {}.\nList of valid options: {}'.format(loss_type, possible_selections))
    elif (isinstance(loss_type, list)) & any(item.lower() not in possible_selections for item in loss_type):
        raise SyntaxError('Provided loss_type argument is not a valid argument: {}.\nList of valid options: {}'.format(loss_type, possible_selections))
    # Raise Type error if there the y_label_full is not provided by the user (Required for normalized errors with IQR present)
    if any(elem is None for elem in y_label_full):
        raise TypeError('an element in y_label_full is None type.\nThis might cause calculation errors for normlalized errors (e.g., nmae).\nTo avoid this problem, please provided the desired y_label data (preferably the full data for IQR normalized errors).')
        
    
    # Convert y_label and y_pred to numpy array if not a numpy array
    if not isinstance(y_label, np.ndarray):
        y_label = np.array(y_label)
    if not isinstance(y_pred, np.ndarray):
        y_pred = np.array(y_pred)
    if not isinstance(y_label_full, np.ndarray):
        y_label_full = np.array(y_label_full)
    
    # Inter quartile range (IQR) (said to be less sensitive to outliers in data), 
    # Explanation: The IQR is a way to measure the spread of the middle 50% of a dataset. 
    # It is calculated as the difference between: 
    # the first quartile* (the 25th percentile) and the third quartile (the 75th percentile) of a dataset.
    if any(y_label_full):
        q3, q1 = np.percentile(y_label_full, [75 ,25])
        iqr = q3 - q1
    
    # *** Perform one of the loss methods from the possible_selections list ***
    try:
        
        losses = {} # empty dictionary to save the loss value or values
        
        # Calculates provided list of loss types and saves all inside a dictionary called losses
        if isinstance(loss_type, list):
            for item in loss_type:
                if item.lower() == 'mse':
                    losses[item.lower()] = np.square(np.subtract(y_label,y_pred)).mean()
                elif item.lower() == 'rmse':
                    losses[item.lower()] = np.sqrt(np.square(np.subtract(y_label,y_pred)).mean())
                elif item.lower() == 'mae':
                    losses[item.lower()]= (np.abs(y_label - y_pred)).mean()
                elif item.lower() == 'mape':
                    losses[item.lower()] = np.mean(np.abs((y_label - y_pred)/y_label))*100
                elif item.lower() == 'nmse':
                    losses[item.lower()] = np.square(np.subtract(y_label,y_pred)).mean()/iqr
                elif item.lower() == 'nrmse':
                    losses[item.lower()] = np.sqrt(np.square(np.subtract(y_label,y_pred)).mean())/iqr
                elif item.lower() == 'nmape':
                    losses[item.lower()] = np.sum(np.abs((y_label - y_pred)))/np.sum((y_label + y_pred))*100
                elif item.lower() == 'nmae':
                    if any(y_label_full):
                        losses[item.lower()] = (np.abs(y_label - y_pred)).mean()/iqr
                elif item.lower() == 'wmape':
                    losses[item.lower()] = np.sum(np.abs((y_label - y_pred)))/np.sum(np.abs(y_label))*100
        # Calculates requested loss type (single value saved inside loss - not a dictionary)
        elif not isinstance(loss_type, list):
            if loss_type.lower() == 'mse':
                losses[loss_type.lower()] = np.square(np.subtract(y_label,y_pred)).mean()
            elif loss_type.lower() == 'rmse':
                losses[loss_type.lower()] = np.sqrt(np.square(np.subtract(y_label,y_pred)).mean())
            elif loss_type.lower() == 'mae':
                losses[loss_type.lower()] = (np.abs(y_label - y_pred)).mean()
            elif loss_type.lower() == 'mape':
                losses[loss_type.lower()] = np.mean(np.abs((y_label - y_pred)/y_label))*100
            elif item.lower() == 'nmse':
                losses[loss_type.lower()] = np.square(np.subtract(y_label,y_pred)).mean()/iqr
            elif item.lower() == 'nrmse':
                losses[loss_type.lower()] = np.sqrt(np.square(np.subtract(y_label,y_pred)).mean())/iqr
            elif item.lower() == 'nmape':
                losses[loss_type.lower()] = np.sum(np.abs((y_label - y_pred)))/np.sum((y_label + y_pred))*100
            elif item.lower() == 'nmae':
                if any(y_label_full):
                    losses[loss_type.lower()] = (np.abs(y_label - y_pred)).mean()/iqr
            elif item.lower() == 'wmape':
                losses[loss_type.lower()] = np.sum(np.abs((y_label - y_pred)))/np.sum(np.abs(y_label))*100
        return losses
    
    except:
        print('An unexpected exception has occured. Check if y_label and y_pred are provided')



def collect_errors(API = None, losses = None):
    
    # Create the API dictionary
    API_DICT = {'API': API}
    
    # Update the given dictionary with incoming losses dictionary
    API_DICT.update(losses)
    
    return API_DICT



In [9]:
def save_pred_results(input_width = None,
                         label_width = None,
                         input_columns = None,
                         norm_method = 'z-score',
                         label_columns = None,
                         batch_number = None,
                         index_slice_start = 0,
                         path_to_save_pred_plots = None, 
                         model = None,
                         model_name = None,
                         index_slice = 'all', 
                         train_df = None, 
                         val_df = None, 
                         test_df = None, 
                         train_mu = None, 
                         train_sigma = None,
                         logscale = 'on',
                         normscale = 'off',
                         plot_loss = 'wmape',
                         split_loss = 'on'):
    
    # Global plot options
    plt.rcParams['figure.figsize'] = [18, 18]
    plt.rcParams.update({'font.size': 20})
    
    # Empty Error list to collect errors
    Error_list = []
    Train_Error_list = []
    Val_Error_list = []
    Test_Error_list = []
    
    # Extract normalized or real labels and predictions for loss computation based on 'normscale' option provided by user
    if normscale == 'on':
        norm_values = '_norm'
    else:
        norm_values = ''
    
    # Check if directory exists and if not create it
    model_type = find_model_type(model = model)

    # Choose mode based on label_width
    if label_width == 1:
        mode = 'many-to-one'
    elif label_width > 1:
        mode = 'many-to-many'

    # Finalize save path
    save_path = os.path.join(path_to_save_pred_plots,mode,*model_type,model_name)

    if not os.path.exists(save_path):
        os.makedirs(save_path)

    # Choose slicing options
    # Full indices to iterate
    if index_slice == 'all':
        total_number_of_wells = len(train_df['API/UWI List'].unique())
        iterable_indices = list(range(0,total_number_of_wells))
    # Partial indices to iterate
    elif isinstance(index_slice, int):
        iterable_indices = list(range(index_slice_start,index_slice))
    
    # Empty list to collect all results to save into excel sheet inside created plots folder
    Forecast_Results = [] # includes train,val,test together
    Forecast_Results_train = []
    Forecast_Results_val = []
    Forecast_Results_test = []
    
    # Prepare to save plots with predictions
    for index,current_well in enumerate(train_df['API/UWI List'].unique()):
        if index in iterable_indices:
            # Get the original data (merged_df)
            df1 = train_df.loc[train_df['API/UWI List'] == current_well]
            df2 = val_df.loc[val_df['API/UWI List'] == current_well]
            df3 = test_df.loc[test_df['API/UWI List'] == current_well]
            current_list = [df1,df2,df3]
            merged_df = pd.concat(current_list)
            
            # Copy of train_df, val_df, test_df to save results for evaluation in three different excel sheets (post-processing)
            train_df_copy = df1[input_width:].copy()
            val_df_copy = df2.copy()
            test_df_copy = df3.copy()
            

            # Get the lenght of train,val,test data
            train_period = len(df1)
            val_period = len(df2) + train_period
            test_period = len(df3) + val_period
            

            # Get predictions
            wpred = WindowGenerator(input_width = input_width, label_width = label_width, shift = 1, input_columns = input_columns, label_columns = label_columns, shuffle_data = False,batch_number = batch_number, train_df=merged_df, val_df=val_df.loc[val_df['API/UWI List'] == current_well], test_df=test_df.loc[test_df['API/UWI List'] == current_well])
            current_forecast = model.predict(wpred.train)

            
            # Save predictions based on fluid phase and mode (many-to-one or many-to-many architecture)
            # Additionally save cumusum of both actual and predicted values in the same manner
            real_forecast = []
            colors = []
            # Get current forecast info and append to real_forecast
            for j,label in enumerate(label_columns):
                if 'Oil'.lower() in label.lower():
                    phase = label.replace('_norm','')
                    
                    # CUMSUM of actual oil phase production (independent of mode type)
                    train_df_copy.insert(len(train_df_copy.keys()), phase + '_CUMSUM',train_df_copy[phase].cumsum(), False)
                    val_df_copy.insert(len(val_df_copy.keys()), phase + '_CUMSUM',val_df_copy[phase].cumsum(), False)
                    test_df_copy.insert(len(test_df_copy.keys()), phase + '_CUMSUM',test_df_copy[phase].cumsum(), False)
                     
                    if mode == 'many-to-one':
                        norm_result = current_forecast[:,j]
                        extract_result = current_forecast[:,j]*train_sigma[index][phase] + train_mu[index][phase]
                        real_forecast.append(extract_result)
                        # Extracting results for train, val, test periods to evaluate
                        train_df_copy.insert(len(train_df_copy.keys()), phase + '_predictions', extract_result[0:train_period-input_width], False)
                        val_df_copy.insert(len(val_df_copy.keys()), phase + '_predictions',extract_result[train_period-input_width:val_period-input_width], False)
                        test_df_copy.insert(len(test_df_copy.keys()), phase + '_predictions', extract_result[val_period-input_width:test_period-input_width], False)
                        train_df_copy.insert(len(train_df_copy.keys()), phase + '_norm' + '_predictions', norm_result[0:train_period-input_width], False)
                        val_df_copy.insert(len(val_df_copy.keys()), phase + '_norm' + '_predictions',norm_result[train_period-input_width:val_period-input_width], False)
                        test_df_copy.insert(len(test_df_copy.keys()), phase + '_norm' + '_predictions', norm_result[val_period-input_width:test_period-input_width], False)
                        # Save CUMSUM (PREDICTIONS) for train, val, test periods to evaluate
                        train_df_copy.insert(len(train_df_copy.keys()), phase + '_CUMSUM' + '_predictions', extract_result[0:train_period-input_width].cumsum(), False)
                        val_df_copy.insert(len(val_df_copy.keys()), phase + '_CUMSUM' + '_predictions',extract_result[train_period-input_width:val_period-input_width].cumsum(), False)
                        test_df_copy.insert(len(test_df_copy.keys()), phase + '_CUMSUM' + '_predictions', extract_result[val_period-input_width:test_period-input_width].cumsum(), False)
                        train_df_copy.insert(len(train_df_copy.keys()), phase + '_norm' + '_CUMSUM' + '_predictions', norm_result[0:train_period-input_width].cumsum(), False)
                        val_df_copy.insert(len(val_df_copy.keys()), phase + '_norm' + '_CUMSUM' + '_predictions',norm_result[train_period-input_width:val_period-input_width].cumsum(), False)
                        test_df_copy.insert(len(test_df_copy.keys()), phase + '_norm' + '_CUMSUM' + '_predictions', norm_result[val_period-input_width:test_period-input_width].cumsum(), False)
                    elif mode == 'many-to-many':
                        if norm_method == 'z-score':
                            norm_result = current_forecast[:,-1][:,j]
                            extract_result = current_forecast[:,-1][:,j]*train_sigma[index][phase] + train_mu[index][phase]
                            real_forecast.append(extract_result)
                        elif norm_method == 'min-max':
                            norm_result = current_forecast[:,-1][:,j]
                            extract_result = current_forecast[:,-1][:,j]*(train_sigma[index][phase] - train_mu[index]) + train_mu[index]
                            real_forecast.append(extract_result)
                            
                        # Extracting results for train, val, test periods to evaluate
                        train_df_copy.insert(len(train_df_copy.keys()), phase + '_predictions', extract_result[0:train_period-input_width], False)
                        val_df_copy.insert(len(val_df_copy.keys()), phase + '_predictions',extract_result[train_period-input_width:val_period-input_width], False)
                        test_df_copy.insert(len(test_df_copy.keys()), phase + '_predictions', extract_result[val_period-input_width:test_period-input_width], False)
                        train_df_copy.insert(len(train_df_copy.keys()), phase + '_norm' + '_predictions', norm_result[0:train_period-input_width], False)
                        val_df_copy.insert(len(val_df_copy.keys()), phase + '_norm' + '_predictions',norm_result[train_period-input_width:val_period-input_width], False)
                        test_df_copy.insert(len(test_df_copy.keys()), phase + '_norm' + '_predictions', norm_result[val_period-input_width:test_period-input_width], False)
                        # Save CUMSUM (PREDICTIONS) for train, val, test periods to evaluate
                        train_df_copy.insert(len(train_df_copy.keys()), phase + '_CUMSUM' + '_predictions', extract_result[0:train_period-input_width].cumsum(), False)
                        val_df_copy.insert(len(val_df_copy.keys()), phase + '_CUMSUM' + '_predictions',extract_result[train_period-input_width:val_period-input_width].cumsum(), False)
                        test_df_copy.insert(len(test_df_copy.keys()), phase + '_CUMSUM' + '_predictions', extract_result[val_period-input_width:test_period-input_width].cumsum(), False)
                        train_df_copy.insert(len(train_df_copy.keys()), phase + '_norm' + '_CUMSUM' + '_predictions', norm_result[0:train_period-input_width].cumsum(), False)
                        val_df_copy.insert(len(val_df_copy.keys()), phase + '_norm' + '_CUMSUM' + '_predictions',norm_result[train_period-input_width:val_period-input_width].cumsum(), False)
                        test_df_copy.insert(len(test_df_copy.keys()), phase + '_norm' + '_CUMSUM' + '_predictions', norm_result[val_period-input_width:test_period-input_width].cumsum(), False)
                    colors.append('g')
                if 'Gas'.lower() in label.lower():
                    phase = label.replace('_norm','')
                    
                    # CUMSUM of actual gas phase production (independent of mode type)
                    train_df_copy.insert(len(train_df_copy.keys()), phase + '_CUMSUM',train_df_copy[phase].cumsum(), False)
                    val_df_copy.insert(len(val_df_copy.keys()), phase + '_CUMSUM',val_df_copy[phase].cumsum(), False)
                    test_df_copy.insert(len(test_df_copy.keys()), phase + '_CUMSUM',test_df_copy[phase].cumsum(), False)
                    
                    if mode == 'many-to-one':
                        norm_result = current_forecast[:,j]
                        extract_result = current_forecast[:,j]*train_sigma[index][phase] + train_mu[index][phase]
                        real_forecast.append(extract_result)
                        # Extracting results for train, val, test periods to evaluate
                        train_df_copy.insert(len(train_df_copy.keys()), phase + '_predictions', extract_result[0:train_period-input_width], False)
                        val_df_copy.insert(len(val_df_copy.keys()), phase + '_predictions',extract_result[train_period-input_width:val_period-input_width], False)
                        test_df_copy.insert(len(test_df_copy.keys()), phase + '_predictions', extract_result[val_period-input_width:test_period-input_width], False)
                        train_df_copy.insert(len(train_df_copy.keys()), phase + '_norm' + '_predictions', norm_result[0:train_period-input_width], False)
                        val_df_copy.insert(len(val_df_copy.keys()), phase + '_norm' + '_predictions',norm_result[train_period-input_width:val_period-input_width], False)
                        test_df_copy.insert(len(test_df_copy.keys()), phase + '_norm' + '_predictions', norm_result[val_period-input_width:test_period-input_width], False)
                        # Save CUMSUM (PREDICTIONS) for train, val, test periods to evaluate
                        train_df_copy.insert(len(train_df_copy.keys()), phase + '_CUMSUM' + '_predictions', extract_result[0:train_period-input_width].cumsum(), False)
                        val_df_copy.insert(len(val_df_copy.keys()), phase + '_CUMSUM' + '_predictions',extract_result[train_period-input_width:val_period-input_width].cumsum(), False)
                        test_df_copy.insert(len(test_df_copy.keys()), phase + '_CUMSUM' + '_predictions', extract_result[val_period-input_width:test_period-input_width].cumsum(), False)
                        train_df_copy.insert(len(train_df_copy.keys()), phase + '_norm' + '_CUMSUM' + '_predictions', norm_result[0:train_period-input_width].cumsum(), False)
                        val_df_copy.insert(len(val_df_copy.keys()), phase + '_norm' + '_CUMSUM' + '_predictions',norm_result[train_period-input_width:val_period-input_width].cumsum(), False)
                        test_df_copy.insert(len(test_df_copy.keys()), phase + '_norm' + '_CUMSUM' + '_predictions', norm_result[val_period-input_width:test_period-input_width].cumsum(), False)
                    elif mode == 'many-to-many':
                        if norm_method == 'z-score':
                            norm_result = current_forecast[:,-1][:,j]
                            extract_result = current_forecast[:,-1][:,j]*train_sigma[index][phase] + train_mu[index][phase]
                            real_forecast.append(extract_result)
                        elif norm_method == 'min-max':
                            norm_result = current_forecast[:,-1][:,j]
                            extract_result = current_forecast[:,-1][:,j]*(train_sigma[index][phase] - train_mu[index]) + train_mu[index]
                            real_forecast.append(extract_result)
                        # Extracting results for train, val, test periods to evaluate
                        train_df_copy.insert(len(train_df_copy.keys()), phase + '_predictions', extract_result[0:train_period-input_width], False)
                        val_df_copy.insert(len(val_df_copy.keys()), phase + '_predictions',extract_result[train_period-input_width:val_period-input_width], False)
                        test_df_copy.insert(len(test_df_copy.keys()), phase + '_predictions', extract_result[val_period-input_width:test_period-input_width], False)
                        train_df_copy.insert(len(train_df_copy.keys()), phase + '_norm' + '_predictions', norm_result[0:train_period-input_width], False)
                        val_df_copy.insert(len(val_df_copy.keys()), phase + '_norm' + '_predictions',norm_result[train_period-input_width:val_period-input_width], False)
                        test_df_copy.insert(len(test_df_copy.keys()), phase + '_norm' + '_predictions', norm_result[val_period-input_width:test_period-input_width], False)
                        # Save CUMSUM (PREDICTIONS) for train, val, test periods to evaluate
                        train_df_copy.insert(len(train_df_copy.keys()), phase + '_CUMSUM' + '_predictions', extract_result[0:train_period-input_width].cumsum(), False)
                        val_df_copy.insert(len(val_df_copy.keys()), phase + '_CUMSUM' + '_predictions',extract_result[train_period-input_width:val_period-input_width].cumsum(), False)
                        test_df_copy.insert(len(test_df_copy.keys()), phase + '_CUMSUM' + '_predictions', extract_result[val_period-input_width:test_period-input_width].cumsum(), False)
                        train_df_copy.insert(len(train_df_copy.keys()), phase + '_norm' + '_CUMSUM' + '_predictions', norm_result[0:train_period-input_width].cumsum(), False)
                        val_df_copy.insert(len(val_df_copy.keys()), phase + '_norm' + '_CUMSUM' + '_predictions',norm_result[train_period-input_width:val_period-input_width].cumsum(), False)
                        test_df_copy.insert(len(test_df_copy.keys()), phase + '_norm' + '_CUMSUM' + '_predictions', norm_result[val_period-input_width:test_period-input_width].cumsum(), False)
                    colors.append('r')
                if 'Water'.lower() in label.lower():
                    phase = label.replace('_norm','')
                    
                    # CUMSUM of actual water phase production (independent of mode type)
                    train_df_copy.insert(len(train_df_copy.keys()), phase + '_CUMSUM',train_df_copy[phase].cumsum(), False)
                    val_df_copy.insert(len(val_df_copy.keys()), phase + '_CUMSUM',val_df_copy[phase].cumsum(), False)
                    test_df_copy.insert(len(test_df_copy.keys()), phase + '_CUMSUM',test_df_copy[phase].cumsum(), False)
                    
                    if mode == 'many-to-one':
                        norm_result = current_forecast[:,j]
                        extract_result = current_forecast[:,j]*train_sigma[index][phase] + train_mu[index][phase]
                        real_forecast.append(extract_result)
                        # Extracting results for train, val, test periods to evaluate
                        train_df_copy.insert(len(train_df_copy.keys()), phase + '_predictions', extract_result[0:train_period-input_width], False)
                        val_df_copy.insert(len(val_df_copy.keys()), phase + '_predictions',extract_result[train_period-input_width:val_period-input_width], False)
                        test_df_copy.insert(len(test_df_copy.keys()), phase + '_predictions', extract_result[val_period-input_width:test_period-input_width], False)
                        train_df_copy.insert(len(train_df_copy.keys()), phase + '_norm' + '_predictions', norm_result[0:train_period-input_width], False)
                        val_df_copy.insert(len(val_df_copy.keys()), phase + '_norm' + '_predictions',norm_result[train_period-input_width:val_period-input_width], False)
                        test_df_copy.insert(len(test_df_copy.keys()), phase + '_norm' + '_predictions', norm_result[val_period-input_width:test_period-input_width], False)
                        # Save CUMSUM (PREDICTIONS) for train, val, test periods to evaluate
                        train_df_copy.insert(len(train_df_copy.keys()), phase + '_CUMSUM' + '_predictions', extract_result[0:train_period-input_width].cumsum(), False)
                        val_df_copy.insert(len(val_df_copy.keys()), phase + '_CUMSUM' + '_predictions',extract_result[train_period-input_width:val_period-input_width].cumsum(), False)
                        test_df_copy.insert(len(test_df_copy.keys()), phase + '_CUMSUM' + '_predictions', extract_result[val_period-input_width:test_period-input_width].cumsum(), False)
                        train_df_copy.insert(len(train_df_copy.keys()), phase + '_norm' + '_CUMSUM' + '_predictions', norm_result[0:train_period-input_width].cumsum(), False)
                        val_df_copy.insert(len(val_df_copy.keys()), phase + '_norm' + '_CUMSUM' + '_predictions',norm_result[train_period-input_width:val_period-input_width].cumsum(), False)
                        test_df_copy.insert(len(test_df_copy.keys()), phase + '_norm' + '_CUMSUM' + '_predictions', norm_result[val_period-input_width:test_period-input_width].cumsum(), False)
                    elif mode == 'many-to-many':
                        if norm_method == 'z-score':
                            norm_result = current_forecast[:,-1][:,j]
                            extract_result = current_forecast[:,-1][:,j]*train_sigma[index][phase] + train_mu[index][phase]
                            real_forecast.append(extract_result)
                        elif norm_method == 'min-max':
                            norm_result = current_forecast[:,-1][:,j]
                            extract_result = current_forecast[:,-1][:,j]*(train_sigma[index][phase] - train_mu[index]) + train_mu[index]
                            real_forecast.append(extract_result)
                        # Extracting results for train, val, test periods to evaluate
                        train_df_copy.insert(len(train_df_copy.keys()), phase + '_predictions', extract_result[0:train_period-input_width], False)
                        val_df_copy.insert(len(val_df_copy.keys()), phase + '_predictions',extract_result[train_period-input_width:val_period-input_width], False)
                        test_df_copy.insert(len(test_df_copy.keys()), phase + '_predictions', extract_result[val_period-input_width:test_period-input_width], False)
                        train_df_copy.insert(len(train_df_copy.keys()), phase + '_norm' + '_predictions', norm_result[0:train_period-input_width], False)
                        val_df_copy.insert(len(val_df_copy.keys()), phase + '_norm' + '_predictions',norm_result[train_period-input_width:val_period-input_width], False)
                        test_df_copy.insert(len(test_df_copy.keys()), phase + '_norm' + '_predictions', norm_result[val_period-input_width:test_period-input_width], False)
                        # Save CUMSUM (PREDICTIONS) for train, val, test periods to evaluate
                        train_df_copy.insert(len(train_df_copy.keys()), phase + '_CUMSUM' + '_predictions', extract_result[0:train_period-input_width].cumsum(), False)
                        val_df_copy.insert(len(val_df_copy.keys()), phase + '_CUMSUM' + '_predictions',extract_result[train_period-input_width:val_period-input_width].cumsum(), False)
                        test_df_copy.insert(len(test_df_copy.keys()), phase + '_CUMSUM' + '_predictions', extract_result[val_period-input_width:test_period-input_width].cumsum(), False)
                        train_df_copy.insert(len(train_df_copy.keys()), phase + '_norm' + '_CUMSUM' + '_predictions', norm_result[0:train_period-input_width].cumsum(), False)
                        val_df_copy.insert(len(val_df_copy.keys()), phase + '_norm' + '_CUMSUM' + '_predictions',norm_result[train_period-input_width:val_period-input_width].cumsum(), False)
                        test_df_copy.insert(len(test_df_copy.keys()), phase + '_norm' + '_CUMSUM' + '_predictions', norm_result[val_period-input_width:test_period-input_width].cumsum(), False)
                    colors.append('b')  
            

            # Append extracted results to evaluate
            Forecast_Results_train.append(train_df_copy)
            Forecast_Results_val.append(val_df_copy)
            Forecast_Results_test.append(test_df_copy)
            
            # Concatonated to compute loss on combined train,val,test per single well
            single_well_results_df = pd.concat([train_df_copy,val_df_copy,test_df_copy])
            Forecast_Results.append(single_well_results_df)
            
            # Number of months in merged df (required for plots)
            total_months = np.array(list(range(0,len(merged_df))))

            # *** Plots and saves graph for predictions for phases in label_columns = ['oil','gas', water] ***
            for i,label in enumerate(label_columns):
                # Current phase to plot
                phase = label.replace('_norm','')
                
                # Available loss types (except mape --> since y_label contains 0 values which causes division issues in this study)
                loss_type = ['mse','rmse','mae','nmse','nrmse','nmape','nmae','wmape','mape'] 
                
                # Train, Val, Test losses per well and per phase
                train_loss = loss_compute(y_label = train_df_copy[phase + norm_values], y_pred = train_df_copy[phase + norm_values + '_predictions'], y_label_full = train_df_copy[phase + norm_values], loss_type = loss_type)
                val_loss = loss_compute(y_label = val_df_copy[phase + norm_values], y_pred = val_df_copy[phase + norm_values + '_predictions'], y_label_full = val_df_copy[phase + norm_values], loss_type = loss_type)
                test_loss = loss_compute(y_label = test_df_copy[phase + norm_values], y_pred = test_df_copy[phase + norm_values + '_predictions'], y_label_full = test_df_copy[phase + norm_values], loss_type = loss_type)
                
                # Calculate mse loss for the well (train,val,test combined)
                single_well_loss = loss_compute(y_label = single_well_results_df[phase + norm_values], y_pred = single_well_results_df[phase + norm_values + '_predictions'], y_label_full = single_well_results_df[phase + norm_values], loss_type = loss_type)
                
                # Collect single well loss
                API_DICT = collect_errors(API = current_well, losses = single_well_loss)
                API_DICT.update({'phase': phase}) 
                Error_list.append(API_DICT)
                
                # Collect train,val,test losses
                Train_API_DICT = collect_errors(API = current_well, losses = train_loss)
                Train_API_DICT.update({'phase': phase}) 
                Train_Error_list.append(Train_API_DICT)
                
                Val_API_DICT = collect_errors(API = current_well, losses = val_loss)
                Val_API_DICT.update({'phase': phase}) 
                Val_Error_list.append(Val_API_DICT)
                
                Test_API_DICT = collect_errors(API = current_well, losses = test_loss)
                Test_API_DICT.update({'phase': phase}) 
                Test_Error_list.append(Test_API_DICT)
                
                # Figure setup
                fig, ax1 = plt.subplots()

                # Plot parameters
                if split_loss == 'off':
                    plt.title((phase + '\n{}: (' + str(round(single_well_loss[plot_loss],3)) + ')').format(plot_loss.upper()), fontweight="bold")
                
                if split_loss == 'on':                
                # Possible plot title change
                    plt.title((phase + '\nTrain {}: (' + str(round(train_loss[plot_loss],3)) + ')' + '\nVal {}: (' + str(round(val_loss[plot_loss],3)) + ')' + '\nTest {}: (' + str(round(test_loss[plot_loss],3)) + ')').format(plot_loss.upper(),plot_loss.upper(),plot_loss.upper()), fontweight="bold", fontsize = 15)

                ax2 = ax1.twinx()
                if logscale == 'on':
                    ax1.set_yscale('log')
                else:
                    ax1.set_yscale('linear')

                # Original Data
                ax1.scatter(total_months,np.array(merged_df[phase]),label=phase, color = 'k')

                # Predictions
                ax1.plot(total_months[input_width:],real_forecast[i], color = colors[i], label = 'Predictions')

                # Plot vertical lines for train,val, and test periods
                ax1.axvline(x = train_period, color = 'orange', linestyle = '--', linewidth = 5, label = 'Train')
                ax1.axvline(x = val_period, color = 'c', linestyle = '--', linewidth = 5, label = 'Validation')
                ax1.axvline(x = test_period, color = 'm', linestyle = '--', linewidth = 5, label = 'Test')

                # Plot control parameter (days on)
                ax2.scatter(total_months[1:],merged_df['Days'][0:-1], s = 96, color = 'deeppink', marker = "x", label = 'Days_on')

                # Label x and y axes
                ax1.set_ylabel('Rate')
                ax1.set_xlabel('Month')

                ax2.set_ylabel('Days on (Control)')

                fig.legend(loc="upper center", fancybox=True, shadow=True, ncol=3)


                # Save and close figures
                plt.savefig(os.path.join(save_path,'API_' + str(current_well) + '_' + phase + '.png'))
                plt.clf()
                plt.close(fig=fig)
    
    # Concatonate all dataframes of all wells (extracted results to evaluate in postprocessing) - saved as an excel sheet
    train_results = pd.concat(Forecast_Results_train)
    val_results = pd.concat(Forecast_Results_val)
    test_results = pd.concat(Forecast_Results_test)
    per_well_results = pd.concat(Forecast_Results)
    
    # Create error dataframe per well (inclused train, val, test)
    Error_data = pd.DataFrame(Error_list)
    Train_Error_data = pd.DataFrame(Train_Error_list)
    Val_Error_data = pd.DataFrame(Val_Error_list)
    Test_Error_data = pd.DataFrame(Test_Error_list)
    
    saved_data = {'train_results':train_results, 
                  'val_results':val_results, 
                  'test_results':test_results,
                  'single_well_results':per_well_results,
                  'single_well_error': Error_data, 
                  'train_error':Train_Error_data, 
                  'val_error':Val_Error_data, 
                  'test_error':Test_Error_data} 
    
    return saved_data

In [10]:
# For easy histogram creation
# Recommended (Example): dataframe = dataframe, column = ['Monthly Oil'], by = ['targetFormation','County'], bins = 'auto', alpha = 0.7, figsize = (15,15) 
def create_hist(dataframe = None, column = None, by = None, bins = None, alpha = None, figsize = None):
    dataframe.hist(column = column, bins=bins, alpha=alpha, by=by, figsize = figsize, edgecolor='black',range=[df[column].min(), df[column].max()] )

In [11]:
# A convenient way to create models and change their hyperparameters (useful for OPTUNA studies)

def create_model(final_layer_return_seq = True, seed = 0, initializer = 'orthogonal', loss='mse', metrics = ['mae','mape'], layers = None, dropout_final_layer = False, dropout_value = 0.2, recurrent_dropout = 'zeros', units = 30, model_type = None, Bi_directional = False, i = None, optimizer = None): #  c = None,
    
    # For globally reproducible random weights
    tf.keras.utils.set_random_seed(seed=seed)
    
    # Initialize weights
    if initializer == 'orthogonal':
        initializer = 'orthogonal'
    elif initializer == 'glorot_uniform':
        initializer = 'glorot_uniform'
    
    # Control recurrent dropout (list of zeros by default)
    if isinstance(recurrent_dropout, str): 
        if ((recurrent_dropout.lower() == 'zeros') or (recurrent_dropout.lower() == 'zero')) and Bi_directional == False:
            recurrent_dropout = [0]*layers
        elif ((recurrent_dropout.lower() == 'zeros') or (recurrent_dropout.lower() == 'zero')) and Bi_directional == True:
            recurrent_dropout = [[0,0]]*layers
        else:
            assert (recurrent_dropout.lower() == 'zeros') or (recurrent_dropout.lower() == 'zero'), 'Provided string is not valid. Only valid option is "zero".'
    elif not isinstance(recurrent_dropout, str):
        recurrent_dropout = recurrent_dropout
        
    # Control units
    assert isinstance(units,list) or isinstance(units, int), 'Only int and list of ints is valid for units variable.'
    if isinstance(units, list):
        assert all(isinstance(element, int) or isinstance(element, list) for element in units), 'Units list contains a string or float element. Units can only be integer elements or a list of integer elements [[int,int]] for Bi-directional RNN models'
        units = units
    elif isinstance(units, int) and Bi_directional == False:
        units = [units]*layers
    elif isinstance(units, int) and Bi_directional == True:
        units = [[units,units]]*layers
    
    # Recalculate layers
    layers = layers - 1
    
    # Main layer
    if model_type == 'GRU':
        main_layer = GRU
    elif model_type == 'LSTM':
        main_layer = LSTM
    
    # if model type is ANN
    if model_type == 'Dense':
        main_layer = Dense
        if layers > 0:
            # Initial Dense layer
            x = main_layer(units = units[0], kernel_initializer=initializer, activation='sigmoid')(i)
            units.pop(0) # remove first element after use

            for num_layers in range(0,layers):
                x = main_layer(units = units[num_layers], kernel_initializer=initializer, activation='sigmoid')(x)

        elif layers == 0:
            # Initial ANN layer
            x = main_layer(units = units[0], kernel_initializer=initializer, activation='sigmoid')(i)
            units.pop(0) # remove first element after use

        # Optional dropout at the end
        if dropout_final_layer == True:
            x = Dropout(dropout_value)(x)

        x = Dense(units=3, activation='sigmoid')(x)

        # Create and compile the model
        model = Model(inputs=i, outputs=x)
        model.compile(optimizer=optimizer, loss = loss, metrics = metrics)
    
    elif model_type == 'LR': # A Linear Regression model using ANN (linear ANN)
        main_layer = Dense
        x = main_layer(units = 3, kernel_initializer=initializer, activation='linear')(i) # directly relate to outputs with no hidden layers when using LR
        units.pop(0) # remove first element after use
        
        # Create and compile the model
        model = Model(inputs=i, outputs=x)
        model.compile(optimizer=optimizer, loss = loss, metrics = metrics)
    
    # if model is an RNN
    elif model_type != 'Dense':

        # Standard GRU or LSTM Option
        if not Bi_directional:
            if final_layer_return_seq == True:
                # Initial RNN layer
                x = main_layer(units = units[0], kernel_initializer=initializer, recurrent_dropout = recurrent_dropout[0], return_sequences=True)(i)
                recurrent_dropout.pop(0) # remove first element after use
                units.pop(0) # remove first element after use

                for num_layers in range(0,layers):
                    x = main_layer(units = units[num_layers], kernel_initializer=initializer, recurrent_dropout = recurrent_dropout[num_layers], return_sequences=True)(x)

            elif final_layer_return_seq == False:
                if layers > 0:
                    # Initial RNN layer
                    x = main_layer(units = units[0], kernel_initializer=initializer, recurrent_dropout = recurrent_dropout[0], return_sequences=True)(i)
                    recurrent_dropout.pop(0) # remove first element after use
                    units.pop(0) # remove first element after use

                    for num_layers in range(0,layers):
                        if (num_layers+1) < layers:
                            x = main_layer(units = units[num_layers], kernel_initializer=initializer, recurrent_dropout = recurrent_dropout[num_layers], return_sequences=True)(x)
                        elif (num_layers+1) == layers:
                            x = main_layer(units = units[num_layers], kernel_initializer=initializer, recurrent_dropout = recurrent_dropout[num_layers], return_sequences=False)(x)

                elif layers == 0:
                    # Initial RNN layer
                    x = main_layer(units = units[0], kernel_initializer=initializer, recurrent_dropout = recurrent_dropout[0], return_sequences=False)(i)
                    recurrent_dropout.pop(0) # remove first element after use
                    units.pop(0) # remove first element after use

            # Optional dropout at the end
            if dropout_final_layer == True:
                x = Dropout(dropout_value)(x)

            x = Dense(units=3, activation='sigmoid')(x)

            # Create and compile the model
            model = Model(inputs=i, outputs=x)
            model.compile(optimizer=optimizer, loss = loss, metrics = metrics)

        # Bi-directional Option (Bi-LSTM or Bi-GRU)
        if Bi_directional:

            # inputs
            inputs = (Input(shape=(i.shape[1], i.shape[2])))

            if final_layer_return_seq == True:
                # forward and backward layers
                forward_layer = main_layer(units = units[0][0], kernel_initializer=initializer, recurrent_dropout = recurrent_dropout[0][0], return_sequences=True)
                backward_layer = main_layer(units = units[0][1], kernel_initializer=initializer, recurrent_dropout = recurrent_dropout[0][1], go_backwards=True, return_sequences=True)
                # Initial RNN layer
                outputs = Bidirectional(layer=forward_layer, backward_layer=backward_layer)(inputs = inputs)
                recurrent_dropout.pop(0) # remove first element after use
                units.pop(0) # remove first element after use

                for num_layers in range(0,layers):
                    outputs = (outputs)
                    forward_layer = main_layer(units = units[num_layers][0], kernel_initializer=initializer, recurrent_dropout = recurrent_dropout[num_layers][0], return_sequences=True)
                    backward_layer = main_layer(units = units[num_layers][1], kernel_initializer=initializer, recurrent_dropout = recurrent_dropout[num_layers][1], go_backwards=True, return_sequences=True)
                    outputs = Bidirectional(layer=forward_layer, backward_layer=backward_layer)(outputs)
            elif final_layer_return_seq == False:
                if layers > 0:
                    # forward and backward layers
                    forward_layer = main_layer(units = units[0][0], kernel_initializer=initializer, recurrent_dropout = recurrent_dropout[0][0], return_sequences=True)
                    backward_layer = main_layer(units = units[0][1], kernel_initializer=initializer, recurrent_dropout = recurrent_dropout[0][1], go_backwards=True, return_sequences=True)
                    # Initial RNN layer
                    outputs = Bidirectional(layer=forward_layer, backward_layer=backward_layer)(inputs = inputs)
                    recurrent_dropout.pop(0) # remove first element after use
                    units.pop(0) # remove first element after use

                    for num_layers in range(0,layers):
                        if (num_layers+1) < layers:
                            outputs = (outputs)
                            forward_layer = main_layer(units = units[num_layers][0], kernel_initializer=initializer, recurrent_dropout = recurrent_dropout[num_layers][0], return_sequences=True)
                            backward_layer = main_layer(units = units[num_layers][1], kernel_initializer=initializer, recurrent_dropout = recurrent_dropout[num_layers][1], go_backwards=True, return_sequences=True)
                            outputs = Bidirectional(layer=forward_layer, backward_layer=backward_layer)(outputs)
                        elif (num_layers+1) == layers:
                            outputs = (outputs)
                            forward_layer = main_layer(units = units[num_layers][0], kernel_initializer=initializer, recurrent_dropout = recurrent_dropout[num_layers][0], return_sequences=False)
                            backward_layer = main_layer(units = units[num_layers][1], kernel_initializer=initializer, recurrent_dropout = recurrent_dropout[num_layers][1], go_backwards=True, return_sequences=False)
                            outputs = Bidirectional(layer=forward_layer, backward_layer=backward_layer)(outputs)

                elif layers == 0:
                    # forward and backward layers
                    forward_layer = main_layer(units = units[0][0], kernel_initializer=initializer, recurrent_dropout = recurrent_dropout[0][0], return_sequences=False)
                    backward_layer = main_layer(units = units[0][1], kernel_initializer=initializer, recurrent_dropout = recurrent_dropout[0][1], go_backwards=False, return_sequences=True)
                    # Initial RNN layer
                    outputs = Bidirectional(layer=forward_layer, backward_layer=backward_layer)(inputs = inputs)
                    recurrent_dropout.pop(0) # remove first element after use
                    units.pop(0) # remove first element after use


            # Optional dropout at the end
            if dropout_final_layer == True:
                outputs = Dropout(dropout_value)(outputs)

            
            # Output layer
            outputs = Dense(units=3, activation='sigmoid')(outputs)
                


            # Create and compile the model
            model = Model(inputs=inputs, outputs=outputs)
            model.compile(optimizer=optimizer, loss = loss, metrics = metrics)

    return model